In [14]:
import requests
import pandas as pd

# Define your FRED API key
API_KEY = "dc6a0313beabf8a2845ed521ddb56d7b"

# Define the FRED API endpoint for FOMC meeting dates
FRED_ENDPOINT = 'https://api.stlouisfed.org/fred/series/observations'

# Parameters for the API request
params = {
    'series_id': 'FEDFUNDS',  # This series represents the federal funds rate, which is influenced by FOMC meetings
    'api_key': API_KEY,
    'file_type': 'json'
}

# Make the API request
response = requests.get(FRED_ENDPOINT, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract observation dates
    observations = data['observations']
    
    
    # Create a DataFrame
    #df = pd.DataFrame(dates, columns=['Date'])
    df = pd.DataFrame(observations)
    df = df[['date', 'value']]
    df.columns = ['date', 'ffr']
    df['date'] = pd.to_datetime(df['date'])
    # Print the DataFrame
    print(df)
else:
    print(f"Error fetching data: {response.status_code}")

# Optionally, save the data to a CSV file
#df.to_csv('fomc_meeting_dates.csv', index=False)


          date   ffr
0   1954-07-01  0.80
1   1954-08-01  1.22
2   1954-09-01  1.07
3   1954-10-01  0.85
4   1954-11-01  0.83
..         ...   ...
835 2024-02-01  5.33
836 2024-03-01  5.33
837 2024-04-01  5.33
838 2024-05-01  5.33
839 2024-06-01  5.33

[840 rows x 2 columns]


In [15]:
df.dtypes

date    datetime64[ns]
ffr             object
dtype: object